<a href="https://colab.research.google.com/github/bpriantti/estudos_quantitative_finances/blob/main/Indice_beta_brunopriantti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0 - Índice Beta:
---
__Objetivo:__   

O presente script demonstra o desenvolvimento em python de tópicos sobre o índice beta, como índice beta, beta rotation e beta setorial.

__Autor:__  
   - Bruno Priantti.
    
__Contato:__  
  - b.priantti@outlook.com

__Encontre-me:__  
   -  www.linkedin.com/in/bpriantti  
   -  https://www.instagram.com/brunopriantti/

___

Biblios. Utilizadas:

- Pandas: https://pandas.pydata.org/

- Numpy: https://numpy.org/

- Matplotlib: https://matplotlib.org/

- Seaborn: https://seaborn.pydata.org/

- Plotly: https://plotly.com/

- Yfinance: https://pypi.org/project/yahoo-finance/

- Statsmodels: https://www.statsmodels.org/stable/index.html

# 1.0 - importando biblios/ dados/ wralling:

In [155]:
#!pip install yfinance

#libs para wralling dos dados:
import pandas as pd
import numpy as np
import statsmodels.api as sm

#libs para visualization dos dados:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as Dash

#lib para api com ativos da bolsa:
import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

# Carrega a base

#ativos:
ticker = ['PETR4.SA', '^BVSP']

#datas - atentar para inicio em janeiro do ano de inicio do ativo ou proximo.
start =  "2000-01-01" 
end =    "2022-12-31"

#API - yahoo finance:
data = yf.download(ticker, start, end)

#wraling:
wralling_functions = [data.isna().sum(),data.isnull().sum(),data.dropna()]
  
for i in wralling_functions:
    data = i

data = data['Adj Close']

data.head()

[*********************100%***********************]  2 of 2 completed


,PETR4.SA,^BVSP
Date,,
2000-01-03,2.627552,16930.0
2000-01-04,2.482198,15851.0
2000-01-05,2.457152,16245.0
2000-01-06,2.448655,16107.0
2000-01-07,2.459836,16309.0


# 2.0 - calculando retornos diarios:

In [156]:
#retornos dos indices:

#ativo:
data['log_ret_atv'] = data.iloc[0:,0].apply(lambda x:np.log(x))
data['log_ret_atv'] = data['log_ret_atv'].diff()

#indice:
data['log_ret_ind'] = data.iloc[0:,1].apply(lambda x:np.log(x))
data['log_ret_ind'] = data['log_ret_ind'].diff()

#removendo nan's
data = data.dropna()

# 3.0 - funcao scatter plot + lin reg:

In [176]:
def beta_plot(start,end, data):
  labels={"log_ret_ind": f"{ticker[1]}"
        ,"log_ret_atv": f"{ticker[0]}"}


  fig = px.scatter(data, x="log_ret_ind", y="log_ret_atv",trendline="ols", trendline_color_override="red", labels = labels)

  fig.update_layout(height=600, width=800,
                    title_text= f"Índice Beta {ticker[1]} vs {ticker[0]}, {start[0:4]} a {end[0:4]}")

  #resultados regressao:
  results = px.get_trendline_results(fig)

  #sets:
  sets = {'font':{'color':'black','size':15},
          'x' : 0.0,
          'y' : -0.12,
          'showarrow' : False,
          'text' : f'Beta: {round(results.px_fit_results.iloc[0].params[1],2)}  R2: {round(results.px_fit_results.iloc[0].rsquared,2)}',
          'textangle' : 0,
          'xanchor' : 'left',
          'xref' : "paper",
          'yref' : "paper"}

  #sets:
  set_logo = {'font':{'color':'black','size':15},
          'x' : 0.85,
          'y' :-0.12,
          'showarrow' : False,
          'text' : f'@brunopriantti',
          'textangle' : 0,
          'xanchor' : 'left',
          'xref' : "paper",
          'yref' : "paper"}

  # add annotation
  fig.add_annotation(sets)
  fig.add_annotation(set_logo)
  fig.show()
 

# 3.0 - visualizando grafico de dispersao + beta:

In [177]:
#print beta rotation!
beta_plot(start,end, data[start:end])

# 4.0 - beta rotation:

In [181]:
lista = []

for x in range(2000,2023,1):

#calculo primeira parte do ano:
  start = f'{x}-01-01'
  end   = f'{x}-06-30'

  X = data[start:end]['log_ret_ind']
  y = data[start:end]['log_ret_atv']

  X = sm.add_constant(X, prepend=False)

  # Fit and summarize OLS model:
  mod = sm.OLS(y, X)
  res = mod.fit()

  #adiciona na lista:
  lista.append(res.params[0])

#calculo segunda parte do ano:
  start = f'{x}-06-30'
  end   = f'{x}-12-31'

  X = data[start:end]['log_ret_ind']
  y = data[start:end]['log_ret_atv']

  X = sm.add_constant(X, prepend=False)

  # Fit and summarize OLS model:
  mod = sm.OLS(y, X)
  res = mod.fit()

  #adiciona na lista:
  lista.append(res.params[0])

#print beta rotation:
dict = {'beta_rotation':lista}
beta_rotation =  pd.DataFrame(dict)
fig = px.line(beta_rotation,  y="beta_rotation", title='Beta Rotation - Semestral 2000 a 2022')
#sets:
set_logo = {'font':{'color':'black','size':15},
        'x' : 0.85,
        'y' :-0.25,
        'showarrow' : False,
        'text' : f'@brunopriantti',
        'textangle' : 0,
        'xanchor' : 'left',
        'xref' : "paper",
        'yref' : "paper"}

# add annotation
fig.add_annotation(set_logo)
fig.update_layout(height=400, width=800)
fig.show()

#5.0 - beta setorial

- carregando dados das etfs e indice SPX:

In [182]:
#ativos:
ticker = ['RCD','RGI','RHS','RTM','RYE','RYF','RYH','RYT','RYU','^GSPC']

#datas - atentar para inicio em janeiro do ano de inicio do ativo ou proximo.
start =  "2000-01-01" 
end =    "2022-12-31"

#API - yahoo finance:
data = yf.download(ticker, start, end)

#wraling:
wralling_functions = [data.isna().sum(),data.isnull().sum(),data.dropna()]
  
for i in wralling_functions:
    data = i

data = data['Adj Close']

data.head()

[*********************100%***********************]  10 of 10 completed


,RCD,RGI,RHS,RTM,RYE,RYF,RYH,RYT,RYU,^GSPC
Date,,,,,,,,,,
2006-11-07,37.555355,39.128395,34.847542,37.555008,37.756649,25.824572,46.498711,43.381519,32.946495,1382.839966
2006-11-08,37.555355,39.128395,34.847542,37.555008,37.756649,25.840387,45.632652,43.381519,32.946495,1385.719971
2006-11-09,37.555355,39.198967,34.847542,38.012794,39.086430,25.676901,45.632652,43.532417,32.946495,1378.329956
2006-11-10,37.770340,39.198967,34.927742,37.486328,38.317780,25.729641,45.070671,43.594566,33.157269,1380.900024
2006-11-13,37.886093,39.371471,34.927742,37.486328,38.317780,25.908943,45.254929,44.207066,33.289764,1384.420044


- calculando retornos e indice beta:

In [188]:
# calculando retorno indice:
data['log_ret_ind'] = data['^GSPC'].apply(lambda x:np.log(x))
data['log_ret_ind'] = data['log_ret_ind'].diff()

#removendo coluna do indice:
#ticker.remove('^GSPC')

#calculando o retorno dos setores:
for x in ticker:
  data[f'return_{x}'] = data[f'{x}'].apply(lambda x:np.log(x))
  data[f'return_{x}'] = data[f'return_{x}'].diff()
 
#removendo nan's
data = data.dropna()

#calculando indice beta:
beta_sect = []

for x in ticker:
  beta_sect.append(np.cov(data[f'return_{x}'],data['log_ret_ind'])[0][1]/np.var(data['log_ret_ind']))

#lista de etfs
etfs = {
    'Consumer Discret':'RCD',
    'Industrials':'RGI',
    'Consumer Staples':'RHS',
    'Materials':'RTM',
    'Energy':'RYE',
    'Financials':'RYF',
    'Heath Care':'RYH',
    'Technology':'RYT',
    'Utilities':'RYU'
}

#viusalizando em barplot:
valores = {
    'names':etfs.keys(),
    'beta_value':beta_sect
}

bar_sect = pd.DataFrame(valores)

bar_sect['beta_value'] = round(bar_sect['beta_value'],2)

fig = px.bar(bar_sect, x='beta_value', y='names', orientation='h',text = 'beta_value',
            title="Beta Setorial Índice S&P 500")

#sets:
set_logo = {'font':{'color':'black','size':15},
        'x' : 0.85,
        'y' :-0.12,
        'showarrow' : False,
        'text' : f'@brunopriantti',
        'textangle' : 0,
        'xanchor' : 'left',
        'xref' : "paper",
        'yref' : "paper"}

# add annotation
fig.add_annotation(set_logo)

fig.update_layout(height=600, width=800)
fig.show()